<h1><font color=red>Créer du HTML avec Python</font></h1>

<h2><font color=red>Lancer le serveur</font></h2/>

L'objectif est de pouvoir créer du code HTML à partir de script Python.

Plus précisemment, nous allons lancer un serveur web écrit en Python qui agira en local (`localhost` ) sur un port précisé dans le script ( ici le port `7777` ).

Pour des raisons de commodité, nous lancerons le script `serveur.py` depuis l'éditeur Spyder, avec la commande `Serveur_Start(7777)` exécutée dans le shell IPython.
Pour arrêter le script, un simple `CTRL+C` dans le shell doit fonctionner.

Vous pouvez alors dans votre navigateur ouvrir un onglet et vous rendre à l'adresse [http://localhost:7777](http://localhost:7777) - et obtenir pour l'instant une erreur 403...

En réalité, tout fichier situé **dans le même répertoire** que le fichier `serveur.py` peut alors être appelé via l'intermédiaire du navigateur, et un fichier en `.py` sera interprété par le serveur. Si ce fichier est construit pour générer du code HTML, le serveur construira ce code puis l'enverra au navigateur, qui affichera une page HTML.

<h2><font color=red>Une première page</font></h2/>

Regardons maintenant une page web simple, nommée `test.py` possédant un formulaire avec deux champs `nom` et `prenom`, page qui renvoie vers elle-même et affiche le nom de l'utilisateur. 
On peut accéder à cette page grâce au navigateur en utilisant cette url [http://localhost:7777/test.py](http://localhost:7777/test.py) Le code de la page est expliqué ci-dessous.

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*

Ces deux lignes dites * magiques* indiquent
* que le langage utilisé est Python3 ;
* que le texte a été encodé en `UTF8`.

In [ ]:
import cgi
formulaire = cgi.FieldStorage() 

On importe le module `cgi` (**Common Gateway Interface** -  Interface de passerelle commune )qui sera quasiment toujours utilisé lorsqu'on souhaite demander au serveur de fair appel à un programme externe plutôt que de déployer du texte html brut. Ce module est celui qui permet de gérer la relation entre la page `test.py` et le serveur web.

Ensuite, une variable nommée `formulaire` est créée, et contient les données éventuellement saisies dans un formulaire précédemment rempli.

In [ ]:
print("Content-type: text/html; charset=utf-8\n")
if formulaire.getvalue("nom")!=None :
    nom=formulaire.getvalue("nom")
    nom=nom.capitalize()
    
else :
    nom="Inconnu"
if formulaire.getvalue("prenom")!=None :
    prenom=formulaire.getvalue("prenom")
    prenom=prenom.capitalize()
    
else :
    prenom=""
   

Ici on regarde si les valeurs saisies dans les formulaires existent avec la comparaison  `!=None`.
Si les valeurs existent, elles sont stockées dans des variables `nom` et `prenom`, puis formatées grâce à la méthode `capitalize()`.
Dans le cas où elles n'existent pas, des valeurs par défaut sont fournies.

In [ ]:
html = """<!DOCTYPE html>
<head>
    <meta charset="utf-8" />
    <title>Bienvenue {nom_form}</title>
</head>
<body >
    <h1 >Bonjour {nom_form}</h1>
    <p >Bienvenue sur cette page, vous êtes {prenom_form} {nom_form}</p>
    <form action="/test.py" method="post">
    <p>Votre nom :<input type="text" name="nom" value="" /></p>
    <p>Votre prénom :<input type="text" name="prenom" value="" /></p>
    <p><input type="submit" name="send" value="Envoyer information au serveur"></p>
</form> 
</body>
</html>
""".format(nom_form=nom, prenom_form=prenom)

print(html)

Enfin, une variable `html` ( mais qui aurait pu s'appeler autrement ) est remplie avec un texte **HTML** sur plusieurs lignes ( à l'aide des triples guillemets ), lui-même formaté de la manière suivante :
* des champs sont sont proposés avec la syntaxe `{nom_form}` et `{prenom_form}`. Ce sont les emplacements dans le code HTML des propositions variables ;
* le texte est formaté grâce à la méthode `format()`, les arguments passés précisant que le champ `nom_form` doit être complété avec la valeur de la variable `nom` ;
* quant au formulaire lui-même, il sera transmis avec la méthode `post` sur la page elle-même, i.e. les données du formulaire sont encalpsulée dans un *dictionnaire*, ici contenant deux entrées `nom` et `prenom` auquel on associe les valeurs saisies par l'utilisateur.

<h2><font color=red>Ajouter le CSS</font></h2>

**<font color=red> Attention, cette méthode n'est pas fondamentalement correcte.</font>**
Pour des raisons de trop grande simplicité du serveur python, il est impossible de faire appel à un fichier CSS avec la méthode habituelle.

Mais il existe un moyen de contourner ce blocage: il s'agit de recopier l'intégralité du fichier CSS à l'intérieur du code HTML. Pour ce faire, à l'intérieur du fichier `test.py` :
1. Ajouter en import le fichier `Serveur`. Celui-ci contient une méthode `Import_CSS()`.
2. La ligne précédant la variable `html`, insérer la ligne de code suivante : `CSS=Serveur.Import_CSS("test_css.css")`.  cette instruction permet à la méthode `Import_CSS()` d'importer sous la forme d'une chaine de caractère le contenu du fichier `test_css.css`, cette chaine de caractère étant stockée dans la variable `CSS`.
3. Il faut alors ajouter cette chaine de caractère à la bonne place, c'est à dire entre les balises `<head>` et `<\head>` de la varaible html. Pour ce faire, on inclut les ligne suivantes :

 `<style>`
 `{CSS_form}`
 `</style>`

4. Reste à formater correctement le texte, en insérant en argument supplémentaire de la méthode `format()` la liaison `CSS_form=CSS`.

Encore une fois, cette méthode est un *bidouillage* propre à cet exercice. Avec un serveur web bien configuré ( type Apache ou autre... ), la balise `<link>` ici inopérante est totalment fonctionnelle, et on peut alors bien séparer la forme ( CSS) du fond (HTML).

<h2><font color=red>Intéragir avec la base de donnée</font></h2>


La dernière chose à comprendre est comment faire intéragir Python avec la base de donnée afin de **créer un code HTML** valide.
Donnons d'abord un exemple de création de formulaire avec liste déroulante puis expliquons le : [http://localhost:7777/test_formulaire_sql.py](http://localhost:7777/test_formulaire_sql.py)

Ici le script python va, en plus de ce qui a été expliqué précédemment, aller chercher dans la base de données `Star_Wars.sql` la liste de tous les titres contenus dans le champ `Titre` de la table `Films`, et pour chacun de ces titres, va créer l'option correspondante de la balise `SELECT`, et *concaténer* le tout sous la forme d'une chaine de caractère.

Analysons le code :

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

print("Content-type: text/html; charset=utf-8\n")##Cette ligne est nécesaire 
#pour faire comprendre au navigateur qu'il s'agit d'un fichier HTML

Comme pour tout script Python devant être exécuté, il est fortement recommandé de donner les deux premières lignes, à savoir le *shebang* ( qui indique l'emplacement logique de la commande `python3` dans une architecture `Linux/Unix`), et l'encodage du fichier.

L'instruction `print()` quant à elle écrit en entête du fichier transmis au navigateur la nature du fichier, à savoir du texte HTML encodé en UTF-8, afin que celui-ci sache quoi faire avec ( c'est-à-dire l'afficher, et non pas le mettre en téléchargement par exemple).

In [ ]:
import cgi, Serveur, sqlite3

On importe les modules nécessaires, à savoir :
* `cgi` pour traiter avec la serveur via la `Common Gateway Interface` ;
* `Serveur` afin d'importer la fonction `Import_CSS()` ;
* `sqlite3` pour communiquer avec la base de donnée.

In [ ]:
##traitement d'un formulaire 
formulaire = cgi.FieldStorage()
if formulaire.getvalue("film")!=None :
    choix_precedent="<p> Vous avez précédemment choisi le film {titre} !</p>".format(titre=formulaire.getvalue("film"))
else :
    choix_precedent="<p> Vou n'avez pas encore choisi de film !</p>"

Avant toute autre opération, on récupère les données précédemment envoyées ( ou non ) par l'intermédiaire d'un formulaire et de la méthode `POST`.
Deux cas se présentent :
* soit le champ `film` du formulaire a été rempli ( `!= None`), c'est-à-dire que quelqu'un a déjà sélectionné dans la liste déroulante un film ), dans ce cas on crée une variable `choix_precedent` à laquelle on affecte une chaine de caractères contenant le nom de la sélection ;
* soit le champ `film` n'existe pas - typiquement lors d'un premier appel à la page, auquel cas la variable `choix_precedent` contient un message adéquat.

In [ ]:
##Import du fichier CSS
CSS=Serveur.Import_CSS("test_css.css")

Conversion du fichier `test_css.css` en une chaine de caractère affectée à une variable `CSS`.

In [ ]:
##traitement des appels à la base de donnée afin de préparer une chaine de 
#caractères formant un formulaire avec
#liste déroulante des films de la bdd
liste_choix=""
connexion=sqlite3.connect('StarWars.sql3')
curs=connexion.cursor()
Titres_extraits=curs.execute("SELECT Titre FROM 'Films';")
for Titre in Titres_extraits :
    liste_choix="\n".join([liste_choix,"<option>{titre}</option>\n"
                               .format(titre=Titre[0])])
curs.close()
connexion.close()

Dans cette partie, on va ouvrir la base de données, puis envoyer une requête de sélection pour obtenir les contenus du champ `Titre` de la table `Films`.
Une fois ces résultats obtenus, on extrait les différents titres puis on les insère dans une chaine de caractère, un par un, entre les balises `<options>` et `</options>`,chaine concaténée au fur et à mesure grâce à la méthode `join()` et le séparateur retour-chariot `\n`.

In [ ]:
##Squelette HTML complété en utilisant la méthode format
html = """<!DOCTYPE html>
<head>
    <meta charset="utf-8" />
    <title>Bienvenue</title>
    <style>
    {CSS}
    </style>
</head>
<body >
    <h1 >Bonjour </h1>
    {choix_precedent}
    <p >Choisissez un des films</p>
    <form action="/test_formulaire_sql.py" method="post">
    <SELECT name="film" size="1">
    {liste_choix_form}
    </SELECT>
    
    <p><input type="submit" name="send" value="Envoyer"></p>
    
</form> 
</body>
</html>
""".format(CSS=CSS,liste_choix_form=liste_choix,choix_precedent=choix_precedent)

print(html)

Enfin, on construit le squelette HTML contenant le texte fixe, puis on complète les trous à l'aide de la méthode `format()`.

<h4><font color='blue'>Exercice</font></h4>

Modifier le fichier `test_formulaire_sql.py` de manière à faire apparaître dans le formulaire une liste déroulante supplémentaire avec le nom des acteurs saisis dans la table `Acteurs` de la base de donnée.